In [1]:
from dotenv import load_dotenv
load_dotenv("../.env")

True

In [2]:
from langgraph.graph import StateGraph, END, START
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import init_chat_model
from langgraph.types import interrupt

import os
project_root = os.path.dirname(os.path.abspath(os.getcwd()))
os.chdir(project_root)

In [3]:
# tools
from tools.tools import send_email_tool
# schemas
from py_files.schemas import EmailFetchOutput, ParsedPrompt, ParsedEditRequest, EmailReply, UserChoice
# states
from py_files.states import EmailAgentState, EmailAgentState2
# functions 
from py_files.functions import parse_prompt, parse_edit_request, generate_email_reply, should_edit, categorize_user_response

In [4]:
# 노드들
def get_email_prompt_node(state : EmailAgentState2) -> EmailAgentState2:
    """서치한 이메일 + 프롬프트 저장"""
    try:
        fetched_email = state["fetched_email"]
    except Exception as e:
        print(f"❌ 이메일 가져오기 오류: {str(e)}")
    
    message = f"""어떤 내용의 답장을 작성하고 싶으신가요?"""
    prompt = input(message).strip()
    return {
        "fetched_email" : fetched_email,
        "prompt" : prompt,
        "parsed_prompt" : None,
        "reply_draft" : None,
        "status" : "RECEIVED_PROMPT"
    }

def parse_prompt_node(state : EmailAgentState2) -> EmailAgentState2:
    """프롬프트 파싱 노드드 (이미 프롬프트를 받았다고 생각)"""
    try:
        parsed_prompt = parse_prompt(state["prompt"])
        return {**state, "parsed_prompt" : parsed_prompt, "status" : "PARSED_PROMPT"}
    except Exception as e:
        print(f"❌ 프롬프트 파싱 오류: {str(e)}")
        return {**state, "status" : "ERROR", "error" : str(e)}
    
def generate_email_reply_node(state : EmailAgentState2) -> EmailAgentState2:
    """첫 번째 답장 생성 노드"""
    previous_reply = 'None'
    try:
        reply_draft = generate_email_reply(state["fetched_email"], previous_reply, state["parsed_prompt"])
        return {**state, "reply_draft" : reply_draft, "status" : "FIRST_DRAFT_GENERATED"}
    except Exception as e:
        print(f"❌ 첫 번째 답장 생성 오류: {str(e)}")
        return {**state, "status" : "ERROR", "error" : str(e)}

def edit_with_feedback_node(state : EmailAgentState2) -> EmailAgentState2:
    """사용자 피드백을 반영해서 수정하는 노드"""
    try:
        # 프롬프트 업데이트, 파싱
        parsed_edit_request = parse_edit_request(state["parsed_prompt"], state["edit_request"])
        # 수정해서 답장 생성
        reply_draft = generate_email_reply(state["fetched_email"], state["reply_draft"], parsed_edit_request)
        return {**state, "parsed_edit_request" : parsed_edit_request, "reply_draft" : reply_draft, "status" : "EDIT_REQUEST_PARSED"}
    except Exception as e:
        print(f"❌ 답장 수정 중 오류: {str(e)}")
        return {**state, "status" : "ERROR", "error" : str(e)}

def generate_email_feedback_node(state : EmailAgentState2) -> EmailAgentState2:
    """답장 생성 결과 피드백 노드"""
    # 답장 생성 결과가 없다면..
    if "reply_draft" not in state:
        print("답장 생성 결과가 없습니다..")
        return {**state, "status" : "ERROR", "error" : "No email reply generated"}
    
    reply_draft = state["reply_draft"]
    print("\n" + "="*50)
    print("📧 [사용자 확인 단계]")
    print("="*50)

    message = f"""
작성한 초안:
--------------------------------
제목 : {reply_draft.subject}
받는 사람 : {reply_draft.to}
내용 : {reply_draft.content}
--------------------------------

초안이 마음에 드시나요? 
- '예' 또는 '확인': 답장 초안이 마음에 듭니다
- 그 외: 답장의 구체적인 수정사항을 말씀해주세요
  (예: "더 친절한 말투로", "제목을 바꿔줘" 등)

입력: """

    user_response = input(message).strip()
    
    print(f"\n사용자 입력: '{user_response}'")
    print("사용자 응답 분석 중...")
    try:
        user_choice = categorize_user_response(user_response)

        if user_choice.kind == "CONFIRM":
            print("확인 - 답장 초안대로 진행하겠습니다!")
            return {**state, "user_feedback" : user_choice, "status" : "RECEIVED_FEEDBACK"}
        else:
            print("수정 요구 - 답장 초안을 수정하겠습니다..")
            return {**state, "user_feedback" : user_choice, "edit_request" : user_response, "status" : "RECEIVED_FEEDBACK"}
    
    except Exception as e:
        print(f"오류 발생: {str(e)}")
        print("기본값 - 답장 초안대로 진행하겠습니다..")
        user_choice = UserChoice(kind = "CONFIRM", goto = "email_responder")

        return {**state, "user_feedback" : user_choice, "status" : "RECEIVED_FEEDBACK"}

def completed_node(state : EmailAgentState2) -> EmailAgentState2:
    """완료 노드"""
    print("\n🎉 이메일 답장 작성이 성공적으로 완료되었습니다!")
    return {**state, "status" : "COMPLETED"}

def error_node(state : EmailAgentState2) -> EmailAgentState2:
    """오류 노드"""
    print("\n❌ 이메일 답장 작성 중 오류가 발생했습니다.")
    print(f"오류 메시지: {state['error']}")

def email_sender_node(state : EmailAgentState2) -> EmailAgentState2:
    """이메일 보내기 노드"""
    try:
        to = state["reply_draft"].to
        subject = state["reply_draft"].subject
        text = state["reply_draft"].content
        send_email_tool.invoke({"to": to, "subject": subject, "text": text})
    except Exception as e:
        print(f"❌ 이메일 보내기 중 오류: {str(e)}")
        return {**state, "status" : "ERROR", "error" : str(e)}
    print("\n🎉 이메일 보내기 완료!")
    return {**state, "status" : "EMAIL_SENT"}

In [5]:
def email_responder_agent():
    """이메일 답장 생성 에이전트"""
    workflow = StateGraph(EmailAgentState2)

    # 노드 추가
    workflow.add_node("get_email_prompt", get_email_prompt_node)
    workflow.add_node("parse_prompt", parse_prompt_node)
    workflow.add_node("generate_email_reply", generate_email_reply_node)
    workflow.add_node("edit_with_feedback", generate_email_feedback_node) # 피드백 받는 노드
    workflow.add_node("feedback_edit", edit_with_feedback_node) # 피드백 기반 수정 노드
    workflow.add_node("completed", completed_node)
    workflow.add_node("error", error_node)
    workflow.add_node("email_sender", email_sender_node)

    # 엣지 추가 
    workflow.add_edge("get_email_prompt", "parse_prompt")
    workflow.add_edge("parse_prompt", "generate_email_reply")
    workflow.add_edge("generate_email_reply", "edit_with_feedback")

    workflow.add_conditional_edges(
        "edit_with_feedback",
        should_edit,
        {
            "completed" : "completed",
            "feedback_edit" : "feedback_edit",
            "error" : "error"
        }
    )
    workflow.add_edge("feedback_edit", "edit_with_feedback")
    workflow.add_edge("completed", "email_sender")
    workflow.add_edge("email_sender", END)
    workflow.add_edge("error", END)

    workflow.set_entry_point("get_email_prompt")

    return workflow.compile()    

In [6]:
def save_agent_diagram():
    """에이전트 워크플로우를 PNG 이미지로 저장"""
    
    agent = email_responder_agent()
    
    try:
        # PNG 이미지로 저장
        from IPython.display import Image, display
        
        # 이미지 생성 및 저장
        img_data = agent.get_graph().draw_mermaid_png()
        
        # 파일로 저장
        with open("email_agent_workflow.png", "wb") as f:
            f.write(img_data)
        
        print("✅ 워크플로우 다이어그램이 'email_agent_workflow.png'로 저장되었습니다!")
        
        # Jupyter에서 이미지 표시
        display(Image(img_data))
        
        return img_data
        
    except ImportError:
        print("❌ 이미지 생성을 위해 추가 패키지가 필요합니다:")
        print("pip install pygraphviz")
        return None
    except Exception as e:
        print(f"❌ 이미지 저장 오류: {str(e)}")
        return None

# 이미지 저장
save_agent_diagram()

❌ 이미지 저장 오류: Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 204.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`


In [ ]:
# ! pip install pyppeteer
# # 또는
! pip install 'langgraph[pyppeteer]'

In [6]:
def save_agent_diagram():
    """에이전트 워크플로우를 PNG 이미지로 저장"""
    
    agent = email_responder_agent()  # 또는 email_fetcher_agent()
    
    try:
        # PNG 이미지로 저장
        from IPython.display import Image, display
        from langgraph.graph.graph import MermaidDrawMethod
        
        # Pyppeteer를 사용한 로컬 렌더링
        img_data = agent.get_graph().draw_mermaid_png(
            draw_method=MermaidDrawMethod.PYPPETEER
        )
        
        # 파일로 저장
        with open("email_agent_workflow.png", "wb") as f:
            f.write(img_data)
        
        print("✅ 워크플로우 다이어그램이 'email_agent_workflow.png'로 저장되었습니다!")
        
        # Jupyter에서 이미지 표시
        display(Image(img_data))
        
        return img_data
        
    except ImportError as e:
        print("❌ Pyppeteer 사용을 위해 추가 패키지가 필요합니다:")
        print("pip install pyppeteer")
        print("또는 다음 명령어로 설치하세요:")
        print("pip install 'langgraph[pyppeteer]'")
        return None
    except Exception as e:
        print(f"❌ 이미지 저장 오류: {str(e)}")
        return None

In [ ]:
npm i -g @mermaid-js/mermaid-cli


In [ ]:
import subprocess, shutil

def save_agent_diagram_mmdc():
    agent = email_responder_agent()
    # 라이브러리마다 mermaid 소스 추출 메서드가 다릅니다.
    # 흔한 케이스: to_mermaid() 또는 draw_mermaid_text() / get_mermaid() 등
    mmd_text = agent.get_graph().to_mermaid()  # 프로젝트에 맞게 메서드명 확인

    with open("email_agent_workflow.mmd", "w", encoding="utf-8") as f:
        f.write(mmd_text)

    if not shutil.which("mmdc"):
        raise RuntimeError("mmdc가 설치되어 있지 않습니다. `npm i -g @mermaid-js/mermaid-cli` 실행 필요")

    # 배경 투명/스케일 등 옵션은 취향대로
    subprocess.run(
        ["mmdc", "-i", "email_agent_workflow.mmd", "-o", "email_agent_workflow.png", "-b", "transparent", "-s", "1.0"],
        check=True
    )
    print("✅ mmdc로 저장 완료: email_agent_workflow.png")


In [10]:
!pip install grandalf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.9 MB/s eta 0:00:00


In [6]:
def quick_fix_diagram():
    """가장 간단한 해결책 - ASCII 다이어그램"""
    
    agent = email_responder_agent()  # 또는 email_fetcher_agent()
    
    try:
        # ASCII 다이어그램 생성
        ascii_diagram = agent.get_graph().draw_ascii()
        
        # 파일로 저장
        with open("email_agent_workflow_ascii.txt", "w", encoding="utf-8") as f:
            f.write(ascii_diagram)
        
        print("✅ ASCII 워크플로우 다이어그램이 저장되었습니다!")
        print("📁 파일: email_agent_workflow_ascii.txt")
        print("\n--- 워크플로우 구조 ---")
        print(ascii_diagram)
        
        return ascii_diagram
        
    except Exception as e:
        print(f"❌ ASCII 다이어그램 생성 실패: {str(e)}")
        return None

# 실행
quick_fix_diagram()

✅ ASCII 워크플로우 다이어그램이 저장되었습니다!
📁 파일: email_agent_workflow_ascii.txt

--- 워크플로우 구조 ---
                          +-----------+                        
                          | __start__ |                        
                          +-----------+                        
                                *                              
                                *                              
                                *                              
                      +------------------+                     
                      | get_email_prompt |                     
                      +------------------+                     
                                *                              
                                *                              
                                *                              
                        +--------------+                       
                        | parse_prompt |                       
                   

'                          +-----------+                        \n                          | __start__ |                        \n                          +-----------+                        \n                                *                              \n                                *                              \n                                *                              \n                      +------------------+                     \n                      | get_email_prompt |                     \n                      +------------------+                     \n                                *                              \n                                *                              \n                                *                              \n                        +--------------+                       \n                        | parse_prompt |                       \n                        +--------------+                       \n                        

In [8]:
def save_agent_diagram_graphviz():
    """Graphviz를 사용해서 다이어그램 저장"""
    
    agent = email_responder_agent()  # 또는 email_fetcher_agent()
    
    try:
        from IPython.display import Image, display
        import io
        
        # Graphviz 방식으로 PNG 생성
        try:
            # draw_ascii() 먼저 시도해서 구조 확인
            ascii_diagram = agent.get_graph().draw_ascii()
            print("📊 워크플로우 구조 (ASCII):")
            print(ascii_diagram)
            
            # PNG 생성 시도
            png_data = agent.get_graph().draw_png()
            
            # 파일로 저장
            with open("email_agent_workflow_graphviz.png", "wb") as f:
                f.write(png_data)
            
            print("✅ Graphviz로 다이어그램이 저장되었습니다!")
            display(Image(png_data))
            return png_data
            
        except Exception as e:
            print(f"❌ Graphviz PNG 생성 실패: {str(e)}")
            print("ASCII 다이어그램만 표시합니다:")
            ascii_diagram = agent.get_graph().draw_ascii()
            print(ascii_diagram)
            return None
            
    except Exception as e:
        print(f"❌ 전체 프로세스 실패: {str(e)}")
        return None

# 실행
save_agent_diagram_graphviz()

📊 워크플로우 구조 (ASCII):
                          +-----------+                        
                          | __start__ |                        
                          +-----------+                        
                                *                              
                                *                              
                                *                              
                      +------------------+                     
                      | get_email_prompt |                     
                      +------------------+                     
                                *                              
                                *                              
                                *                              
                        +--------------+                       
                        | parse_prompt |                       
                        +--------------+                       
                    